In [110]:
import pandas as pd
import numpy as np

In [2]:
positions_c = ['legislature', 'identifiant', 'groupe', 'membres', 'pour', 'contre', 'nonvotants', 'abstention']
votes_c = ['legislature', 'identifiant', 'nom', 'prenom', 'groupe', 'position']

In [18]:
groupes = {
    u'Groupe socialiste, r\xe9publicain et citoyen': u'PS',
    u'Groupe Les R\xe9publicains': u'LR',
    u"Groupe de l'union des d\xe9mocrates et ind\xe9pendants": u'UDI',
    u'Groupe \xe9cologiste': u'Verts',
    u'Groupe radical, r\xe9publicain, d\xe9mocrate et progressiste': u'PRG',
    u'Groupe de la gauche d\xe9mocrate et r\xe9publicaine': u'FdG',
    u'Non inscrits': u'NI',
    u"Groupe de l'union pour un mouvement populaire": u'LR',
    u'Groupe rassemblement-union pour un mouvement populaire': u'UMP-Fillon',
}

In [19]:
pd.set_option('display.max_columns', None)

In [20]:
positions = pd.read_csv('output\\position.csv', sep=';', encoding='cp1252').reindex(columns=positions_c)

In [21]:
positions['groupe'] = positions.groupe.map(groupes)

In [23]:
from functools import reduce

In [24]:
positions['nb_null'] = \
    reduce(pd.Series.__add__, [positions[c].isnull().astype(int) for c in ['abstention', 'pour', 'contre', 'nonvotants']])

In [25]:
positions = positions.sort_values(['legislature', 'identifiant', 'groupe', 'nb_null'])\
                     .drop_duplicates(subset=['legislature', 'identifiant', 'groupe'], keep='first')\
                     .reindex(columns=positions_c)

In [27]:
positions.set_index(['legislature', 'identifiant', 'groupe']).unstack().swaplevel(0, 1, axis=1).sortlevel(axis=1).fillna(0)

groupe                      FdG                                        LR  \
                        membres pour contre nonvotants abstention membres   
legislature identifiant                                                     
14          1                15    3      0          0         12     196   
            2                15    2      0          0          0     196   
            3                15    0      3          0          0     196   
            4                15    0      3          0          0     196   
            5                15    2      0          0          0     196   
            6                15    0      3          0          0     196   
            7                15    2      0          0          0     196   
            8                15    2      0          0          0     196   
            9                15    0      0          0          2     196   
            10               15    3      0          0          0     196   
            11               15    2      0          0          0     196   
            12               15    2      0          0          0     196   
            13               15    0      2          0          0     196   
            14               15    0      3          0          0     196   
            15               15    0      3          0          0     196   
            16               15    0      4          0          0     196   
            17               15    4      0          0          0     196   
            18               15    2      0          0          0     195   
            19               15    0      2          0          0     195   
            20               15    2      0          0          0     195   
            21               15    0      0          0          2     195   
            22               15    0      0          0          0     195   
            23               15    0      0          0          0     195   
            24               15    0      2          0          0     195   
            25               15    0      2          0          0     195   
            26               15    0      2          0          0     195   
            27               15    0      2          0          0     195   
            28               15    0      2          0          0     195   
            29               15    0      2          0          0     195   
            30               15    1     13          0          1     195   
...                         ...  ...    ...        ...        ...     ...   
            1181             15    0      1          0          0     199   
            1182             15    0      0          0          0     199   
            1183             15    0      0          0          0     199   
            1184             15    0      0          0          0     199   
            1185             15    2     12          0          1     199   
            1186             15    0      0          0          0     199   
            1187             15    0      0          0          0     199   
            1188             15    0      0          0          0     199   
            1189             15    2     12          0          1     199   
            1190             15    0      6          0          0     199   
            1191             15   15      0          0          0     199   
            1192             15    1      0          0          0     199   
            1193             15    0      0          0          0     199   
            1194             15    0      1          0          0     199   
            1195             15    1      0          0         10     199   
            1196             15    0      0          0          0     199   
            1197             15    0      0          0          1     199   
            1198             15    0      0          0          0     199

In [28]:
deputes = pd.read_csv('output/depute.csv', sep=';', encoding='cp1252')

In [31]:
deputes['nom'] = deputes.nom.map(lambda s: ' '.join(s.split(' ')[1:]))

In [86]:
votes = pd.read_csv('output/vote.csv', sep=';', encoding='cp1252').reindex(columns=votes_c)

In [87]:
votes['groupe'] = votes['groupe'].map(groupes)
votes['position'] = votes['position'].astype('category', categories=['pour', 'contre', 'non-votant', 'abstention', 'absent'])

In [130]:
import unicodedata
def strip_accents(s):
   return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')


In [150]:
noms = votes[['nom', 'prenom']].drop_duplicates().sort_values(['nom', 'prenom'])

In [151]:
noms_votes = noms['prenom'] + ' ' + noms['nom']

In [152]:
tr = str.maketrans('\xa0-', ' '*2)

In [162]:
transformations = {
    'Genevieve Gosselin': 'Genevieve Gosselin Fleury',
    'Gerard MENUEL': 'Gerard Menuel',
}

In [163]:
noms_votes = set((noms['prenom'] + ' ' + noms['nom']).map(strip_accents).replace(transformations).str.translate(tr))

In [164]:
noms_deputes = set(deputes['nom'].map(strip_accents).str.translate(tr))

In [165]:
noms_votes - noms_deputes

{'Claude Bartolone'}

In [169]:
votes['nom complet'] = (votes['prenom'] + ' ' + votes['nom']).map(strip_accents).replace(transformations).str.translate(tr)

In [171]:
tableau = votes.reindex(columns=['nom complet', 'identifiant', 'position'])\
               .set_index(['identifiant', 'nom complet'])['position']\
               .unstack().fillna('absent')

In [172]:
tableau.to_csv('output/tableau.csv', sep=';', encoding='cp1252')